## AI Sales Agent   
     
We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

Please set up an account      
https://sendgrid.com/       
(Sendgrid is a Twilio company for sending emails.)      
Settings (left sidebar) >> API Keys >> Create API Key (button on top right)      
Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"      
and verify that your own email address is a real email address, so that SendGrid can send emails for you.      
(you can also use sparkpost)      
       
Website domain    
(i.e. https://porkbun.com/)     


#### replace from/to email address before running (r@r)

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

## Step 1: Agent workflow

let's do a simple workflow between agents. Here we're just going to have some agents calling other agents, and we're going to orchestrate it manually ourselves in Python code.         

In [3]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

We create three agents      
working for "ComplAI" soc2 compliance company       
(a cybersecurity compliance framework developed by the American Institute of Certified Public Accountants (AICPA). The primary purpose of SOC 2 is to ensure that third-party service providers store and process client data in a secure manner.)       

In [4]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

We looked at runner.run. Now, runner.run.streamed. This will stream back the results.      
      
There is no "await" keyword that means we are not getting a "response" but a "co-routine"       
We use a special construct "async" will return a "co-routine" and iterate through those answeres.     
Boilerplate code to make sure what is coming back is some text that we can print 

In [5]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with AI-Powered Solutions

Dear [Recipient's Name],

I hope this message finds you well. I'm reaching out to introduce you to ComplAI, an industry leader in simplifying SOC 2 compliance through our innovative SaaS platform.

Navigating the complexities of compliance and audit preparation can be daunting, especially when managing operational efficiency. Our AI-driven tool not only automates key processes but also provides real-time insights that ensure your organization meets rigorous SOC 2 standards.

Key benefits of using ComplAI include:

- **Streamlined Documentation**: Automatically generate and maintain up-to-date documentation required for compliance.
- **Audit Readiness**: Prepare for audits with confidence, reducing time and resources spent on compliance tasks.
- **Real-Time Monitoring**: Gain actionable insights and alerts, helping you stay on top of compliance requirements.

I would love to schedule a brief call to discuss how ComplAI

We are calling "asyncio"       
We are running three different runs at the same time, not multithreading but using an event loop which will run each one when it is pausing/waiting on one.      
Alternate talking to a LLM and collect back the results and what comes back is a collection of results      
      
Also note the "trace" wrapped into one, 

In [6]:
# import asyncio 

message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Simplify Your SOC 2 Compliance Journey with ComplAI

Hi [Recipient's Name],

I hope this message finds you well.

As companies face increasing scrutiny regarding data security, ensuring SOC 2 compliance has become pivotal. However, the complexities of preparing for audits can often consume valuable resources and time.

At ComplAI, we’ve developed a cutting-edge SaaS tool specifically designed to streamline the SOC 2 compliance process. Powered by AI, our platform simplifies the management of documentation, automates evidence collection, and provides real-time insights into your compliance status. This allows your team to focus on what truly matters – growing your business.

Here are a few ways ComplAI can support your compliance efforts:

- **Automated Documentation**: Reduce manual effort and minimize errors with our intuitive documentation tools.
- **Real-Time Monitoring**: Stay ahead of compliance requirements with continuous monitoring and alerts.
- **Audit Preparedness**:

We are creating a new agent "Sales_picker"  to pick the best email

In [7]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
)

We have this same construct, we are going to run three agents in parallel and gather the outputs and then call another agent and select the best email and print    
Again, all under one trace

In [8]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: Make SOC2 Compliance as Easy as Pie (or Cake, Your Pick!)

Hey [Recipient's Name],

Ever felt like achieving SOC2 compliance is akin to deciphering the Rosetta Stone—only with more paperwork and fewer hieroglyphics? 🤔

Well, you're not alone! At ComplAI, we’ve transformed the compliance process into a smooth, breezy ride powered by the magic of AI. 🪄✨

Imagine having a trusty sidekick (no cape required) that not only helps you prepare for audits but also keeps you SOC2 compliant while you focus on what you do best: running your business!

**Here’s what we bring to the table:**
- **Automagical Documentation:** Say goodbye to endless spreadsheets—our tool takes the grunt work off your plate. 🥳
- **Real-time Compliance Monitoring:** No more last-minute panic! We’ll keep an eye on everything, so you don’t have to. ⏰
- **Audit Prep Made Fun:** (Well, as fun as compliance can be!) Our platform will guide you through the audit process with ease.

Interested in a qui

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [9]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

If we look at one agent,       
it's got a name (name='Professional Sales Agent'), instructions (instructions='You are a sales agent ... ). and you can see that it has things like tool choice (tool_choice=None) tools, and it has like a list of tools (tools=[]) that it has access to but right now it is empty. 

In [10]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

This tool/function sends email "send_email". email boyd is a string, doc string. sends out an email with the given body to all sal3s prospects.       
     
from_email, verified sender set up in SendGrid      
to_email, recipient      
      
SendGrid API - sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))       
response = sg.client.mail.send.post(request_body=mail)      
      
add a decorator "@function_tool" that allows OpenAI to write some code with the "send_email" function

In [11]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("r@r.com")  # Change to your verified sender
    to_email = To("r@yahoo.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

It has a name "send_email" and a description that is taken from doc string.      
And then " params_json_schema ..." this is a boilerplate JSON that we were manually adding it previously

In [12]:
# Let's look at it
send_email 

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000214DD7B23E0>, strict_json_schema=True)

### And you can also convert an Agent into a tool

We have already turned a function into a tool.        
Whole agent can also be turned into a tool       

In [13]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000214DD9284A0>, strict_json_schema=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [14]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000214DD8A65C0>, strict_json_schema=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000214DD901DA0>, strict_json_schema=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title'

## And now it's time for our Sales Manager - our planning agent

In [15]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across

### Summary

We first just used agents to write Python code so that agents could run. We sort of used the async gather so that we could run multiple in parallel and then we used another agent afterwards to pick the best email.    
     
We then use tools to wrap a function so that we could have an agent that calls a function and we also used the as tool construct so that we could wrap other agents to be a tool and we provided all of those tools to our sales manager agent and we let it call three different email producers and then finally send the email at the end..       
      
A **hand-off** is something that you can give an agent. An agent has a number of tools, and a number of handoffs, and handoffs are other agents that it can delegate to. So in many ways, that sounds really similar to taking an agent and wrapping it as a tool.      
      
An agent as tool is very similar to a handoff. There's a kind of conceptual difference and and a very practical difference.The conceptual difference is that you can you can sort of think of it like there's an agent which either has just the ability to use tools as part of doing its job, or handoff is when it's delegating. It's sort of giving giving responsibility and ownership of a specialist task to another agent.      
      
But there's a more sort of fundamental, simple technical difference between them. When you're using tools, you can think of that more as a request response. You're calling the tool and control passes back to you, and you continue as the main agent executing      
     
in the case of handoffs. You've done your piece, and you are now passing control to the other agent, and the flow does not come back to you again.       
       


#### And now we're going to make a new agent that eventually is going to be a handoff     
     
An agent to write a subject for emails      
       
Another instruction is to convert a text email to an HTML email. Body of email might have markdown and needs to converted to HTML email with simple, clear, compelling layout and design.    
     
So we're then going to make an agent for the subject writer with those instructions. And we're going to use the as tool approach. This time we're going to make this a tool because it sounds like a tool right. It's just a tool to write a subject.    
      
Similarly for the HTML converter, as a tool.      

In [16]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

And then we are going to have one more tool which sends email with subject and body.    

In [17]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("r@rcom")  # Change to your verified sender
    to_email = To("r@yahoo.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

Three tools, two are agents and one is just a function.           
Agents       
- "Subject Writer"      
- "HTML converter"     
Function        
- "Send html email" 

In [18]:
tools = [subject_tool, html_tool, send_html_email]

In [19]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000214DD901300>, strict_json_schema=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000214DD95A520>, strict_json_schema=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales prospects', params_json_

So now we are going to create a separate agent that we want to hand off to.        
It has hands off description.     

In [20]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

### Now we have 3 tools and 1 handoff 
      
Tools - Three sales agents    
handoffs - this itself has three tools, subject writer, HTML converter and send email 

In [21]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000214DD8A65C0>, strict_json_schema=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000214DD901DA0>, strict_json_schema=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 

New Sales Manager, that generate cols sales emails (read the instructions).  

In [22]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has announced their Agent Development Kit (ADK) which is in early preview. It's still under development, so it's too early for us to cover it here. But it's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!